In [122]:
import os
import torch


# Set GPU device
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


Load Dataset - Alpaca

In [124]:
import pandas as pd

# Load JSON data into DataFrame
df = pd.read_json('./data/alpaca_data_cleaned.json')

In [125]:
df.head()

,instruction,input,output
0,Give three tips for staying healthy.,,1. Eat a balanced and nutritious diet: Make su...
1,What are the three primary colors?,,"The three primary colors are red, blue, and ye..."
2,Describe the structure of an atom.,,An atom is the basic building block of all mat...
3,How can we reduce air pollution?,,There are several ways to reduce air pollution...
4,Pretend you are a project manager of a constru...,,I had to make a difficult decision when I was ...


In [126]:
from datasets import Dataset, DatasetDict

# Function to convert dataframe to datasets object
def dataframe_to_dataset(df):
   
    dataset_dict = {  #dictionary with keys corresponding to column names
        "instruction": df["instruction"],
        "input": df["input"],
        "output": df["output"],
    }
    # Create a datasets object
    dataset = Dataset.from_dict(dataset_dict)
    return dataset

In [127]:
# Convert dataframe to datasets object
dataset = dataframe_to_dataset(df)

# Split the dataset into train and test splits
train_size = int(0.8 * len(dataset))
train_dataset = dataset.select(range(train_size))
test_dataset = dataset.select(range(train_size, len(dataset)))

# Create a DatasetDict with train and test splits
dataset_dict = DatasetDict({"train": train_dataset, "test": test_dataset})

# Print the dataset dictionary
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 41408
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 10352
    })
})


In [ ]:
#Remove input field from dataset as it is empty
dataset_dict['train'] = dataset_dict['train'].remove_columns("input")
dataset_dict['test'] = dataset_dict['test'].remove_columns("input")
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output'],
        num_rows: 41408
    })
    test: Dataset({
        features: ['instruction', 'output'],
        num_rows: 10352
    })
})

### SFT Training

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import SFTTrainer

In [150]:
# Formatting Instruction/Prompt for training

def format_instruction(sample):

	return [f"""
Below is an input that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{sample['instruction']}""",
f"""### Response:{sample['output']}""".strip()]


In [151]:
# Get gpt2 model from huggingface
model = AutoModelForCausalLM.from_pretrained("distilgpt2", device_map = 'auto')
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token

In [152]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output'],
        num_rows: 41408
    })
    test: Dataset({
        features: ['instruction', 'output'],
        num_rows: 10352
    })
})

In [154]:
# Training the Alpaca dataset with SFT Trainer on GPT2 pretrained modl

trainer = SFTTrainer(
    model,
    train_dataset=dataset_dict['train'],
    tokenizer=tokenizer,
    max_seq_length=1024,
    formatting_func=format_instruction,
)

trainer.train()

  0%|          | 0/42 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=33, training_loss=3.033475702459162, metrics={'train_runtime': 51.1238, 'train_samples_per_second': 4.929, 'train_steps_per_second': 0.645, 'total_flos': 65846781149184.0, 'train_loss': 3.033475702459162, 'epoch': 3.0})

### Save the trained model

In [155]:
# Define the directory path where you want to save the model
save_directory = "./models"

# Save the model and tokenizer
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print("Model saved successfully.")


Model saved successfully.


### Loading the saved model

In [156]:
from transformers import AutoModelForCausalLM, AutoTokenizer

saved_model = AutoModelForCausalLM.from_pretrained(save_directory)
saved_tokenizer = AutoTokenizer.from_pretrained(save_directory)

print("Model loaded successfully.")


Model loaded successfully.


### Prepare Evaluation Data

In [157]:
# !git lfs install

In [35]:
# Download evaluation data
# cd data
# !git clone https://huggingface.co/datasets/tatsu-lab/alpaca_eval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Cloning into 'alpaca_eval'...
remote: Enumerating objects: 233, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 233 (delta 7), reused 0 (delta 0), pack-reused 212
Receiving objects: 100% (233/233), 3.37 MiB | 19.19 MiB/s, done.
Resolving deltas: 100% (112/112), done.
Filtering content: 100% (4/4), 67.91 MiB | 50.06 MiB/s, done.


In [190]:
# Creation the dataset object from evaluation data which is in json by conveting into pandas dataframe first

eval_df = pd.read_json('./data/alpaca_eval.json')
dataset_dict = {
    "instruction": eval_df["instruction"],
    "output": eval_df["output"],
}

eval_dataset = Dataset.from_dict(dataset_dict)

In [191]:
eval_dataset

Dataset({
    features: ['instruction', 'output'],
    num_rows: 805
})

### Model Inference

In [192]:
from transformers import pipeline

# Initialize the text generation pipeline
text_generator = pipeline("text-generation", model=saved_model, tokenizer=saved_tokenizer)
# Input prompt
prompt = "Explain why the following fraction is equivalent to 1/4"
# Text Generation
generated_text = text_generator(prompt, max_length=1024, num_return_sequences=1)[0]['generated_text']
# Generated text
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Explain why the following fraction is equivalent to 1/4 of the total number of pixels in a window.


So let us go back to the original equation and consider the ratio of pixels to pixels to pixels.
In our example we get the following number of pixels:
A. If we go from 1/4 of the total number of pixels in a window to 4, this represents the ratio of pixels to pixels for each pixel of the window.
A. It would be a rather difficult calculation in a way if we had gone from 0 to 0. Hence if we go from 1/4 of the total number of pixels in a window, our total number of pixels would be 0.
In the past, it's been suggested that multiplying 1/4 of the total number of pixels in a window would be impossible with just the number of pixels in the window, not necessarily the fact that the same number of pixels in the window would be multiplied. However, we can now be confident that at least the sum of the two values in the window would be correct and that the sum of two values in the window would be cor

In [ ]:
# Function to calculate cosine similarity between two texts

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_cosine_similarity(text1, text2):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([text1, text2])
    cosine_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]
    return cosine_sim


### Comparision with Alpaca Gold Label

In [194]:
# Prepare the input prompts
input_prompts = [f"{example['instruction']}" for example in eval_dataset]

# Generate responses from the model
generated_responses = []
for prompt in input_prompts:
    try:
        encoded_prompt = saved_tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
        output = saved_model.generate(**encoded_prompt)
        generated_response = tokenizer.decode(output[0], skip_special_tokens=True)
        print(generated_response)
        generated_responses.append(generated_response)
    except:
        continue

# Compare generated responses with gold labels
gold_labels = [example["output"] for example in eval_dataset]

# Calculate accuracy or other evaluation metrics
correct_predictions = sum(1 for gen_resp, gold_label in zip(generated_responses, gold_labels) if gen_resp == gold_label)
similar_predictions = sum(calculate_cosine_similarity(gen_resp, gold_label) for gen_resp, gold_label in zip(generated_responses, gold_labels))
absolute_accuracy = correct_predictions / len(eval_dataset)
cosine_accuracy = similar_predictions / len(eval_dataset)
print(f"Absolute Accuracy: {absolute_accuracy}")
print(f"Cosine Accuracy: {cosine_accuracy}")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What are the names of some famous actors that started their careers on Broadway? What are their names?”


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


How did US states get their names?


To understand what to do about the US, please see the following questions: - Can we use your local telephone number to notify you of any issues related to the US Federal Tax Return? - For


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Compose an engaging travel blog post about a recent trip to Hawaii, highlighting cultural experiences and must-see attractions. This is where you start making a conscious decision.


A trip to Hawaii is a worthwhile trip if you have been with Hawaiian


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Write a captivating movie review for a recently released science fiction film, discussing its plot, characters, and special effects.




What is the theme of this film?
The theme of the film is similar to the theme that came


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Structure a podcast script for an episode discussing the influence of streaming platforms on the music industry.




For more on Vimeo, subscribe here.
Write a symphony concert review, discussing the orchestra's performance and overall audience experience.
Absolute Accuracy: 0.0
Cosine Accuracy: 0.09083677505108742
